In [1]:
import pandas as pd
from tqdm.notebook import tqdm


df = pd.read_csv('/Users/riadanas/Desktop/steven_bartlett_project/data/raw/DIARY_all_pod.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
'''
print(df.shape)

# Keep a small sample for testing
df = df.head(500)


print(df.shape)
'''


'\nprint(df.shape)\n\n# Keep a small sample for testing\ndf = df.head(500)\n\n\nprint(df.shape)\n'

In [3]:
print(df.shape)

(402054, 18)


In [4]:


### If I want to pick random samples from a few videos
import random

video_ids = df['video_id'].unique()[5:13]
df = df[df['video_id'].isin(video_ids)].sample(n=100)

print(df.shape)
print(df['video_id'].value_counts())
print(df['video_title'].value_counts())
df.head(2)


(100, 18)
video_id
NyFSkGMWP5Q    38
ldizQkuWpDE    22
Hik6OY-nk4c    15
atejm2w2jWY     7
It5_C6AF1pk     7
ZxXruY7llcc     6
EdlXcVu1CTs     3
0GQozcTPyO0     2
Name: count, dtype: int64
video_title
The Insulin & Glucose Doctor: This Will Strip Your Fat Faster Than Anything!                          38
Body Language Expert Explains Why People Dislike You                                                  22
Jordan B Peterson: You Need To Listen To Your Wife! We've Built A Lonely & Sexless Society!           15
Shaolin Warrior Master: Hidden Epidemic Nobody Talks About! This Modern Habit Is Killing Millions!     7
Exercise & Nutrition Scientist: The Truth About Exercise On Your Period! Take These 4 Supplements!     7
Peter Attia: Anti-aging Cure No One Talks About! 50% Chance You’ll Die In A Year If This Happens!      6
The Business Expert: How To Build A Brand In 2025! They're Lying To You About Work-Life Balance!       3
Hormone Expert: Control Your Hormones Control Your Belly Fat! Co

,channel_name,channel_id,video_id,video_title,video_description,video_published_at,view_count,video_like_count,comment_count,comment_id,comment_text,author,author_id,comment_like_count,comment_published_at,is_pinned,is_reply,parent_comment_id
77400,The Diary Of A CEO,UCGq-a57w-aPwyi3pW7XLiHw,NyFSkGMWP5Q,The Insulin & Glucose Doctor: This Will Strip ...,"88% of adults have metabolic disease, but what...",2025-02-06T08:00:17Z,5387358,162020,10597,UgyU9rxYG4P0NUL8WA54AaABAg.AJJ7oTBGcPJAM8MkWBCtqL,Great to hear about your turn around. Bravo!,@roflorida1,UC7q8r-7ahW4TtkoUziiQoww,0,2025-08-22T19:50:30Z,False,True,UgyU9rxYG4P0NUL8WA54AaABAg
58070,The Diary Of A CEO,UCGq-a57w-aPwyi3pW7XLiHw,Hik6OY-nk4c,Jordan B Peterson: You Need To Listen To Your ...,Dr Jordan Peterson is a world-renowned former ...,2025-01-13T08:00:19Z,2046763,54184,6392,UgyXAbAEOln-oBI6w4h4AaABAg.ADEVDGzRuVDADEiCiG23KF,Not a very open minded view....,@NiniM8154,UCLyDLLBemSNUmPREQpFGr5A,1,2025-01-13T14:19:37Z,False,True,UgyXAbAEOln-oBI6w4h4AaABAg


## Guest Name Processing - GPT-4

In [5]:
import os
import json
import re
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

def get_guest_names_openrouter(description: str) -> list:
    """
    Extract true podcast guest names from a YouTube description using OpenRouter (Claude 3.5 / GPT-4-mini).
    Ignores names used as references or examples.
    """
    if not isinstance(description, str) or not description.strip():
        return []

    prompt = f"""
    You are a podcast metadata assistant.

    Task:
    - Read the YouTube video description carefully.
    - Identify ONLY the actual guest(s) who appear in the episode or are directly interviewed.
    - If a guest's name was misspelled, correct it based on context.
    - Make sure to not miss guests that go by nicknames (e.g., "The Rock" or "MrBeast").
    - Ignore people mentioned just as examples, comparisons, or references (e.g., Warren Buffett, Elon Musk) unless they are clearly stated as guests.
    - If multiple guests appear, include all of them.
    - Preserve professional titles (e.g., "Dr", "Prof", "Sir") if present.
    - Return a clean JSON list of guest names, for example:
      ["Morgan Housel"]
      or ["Dr Andrew Huberman", "Lex Fridman"]
    - If no guest is clearly identified, return an empty list [].

    Description:
    \"\"\"{description}\"\"\"
    """

    try:
        completion = client.chat.completions.create(
            model="openai/gpt-4o-mini",  # you can change to "openai/gpt-4o-mini"
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=200,
        )

        content = completion.choices[0].message.content.strip()

        # Try parsing JSON
        try:
            result = json.loads(content)
        except json.JSONDecodeError:
            match = re.search(r'\[(.*?)\]', content)
            if match:
                inner = match.group(1)
                result = [n.strip().strip('"').strip() for n in inner.split(",") if n.strip()]
            else:
                result = re.findall(r"(?:Dr\.?|Prof\.?|Mr\.?|Ms\.?)?\s?[A-Z][a-z]+(?:\s[A-Z][a-z]+)+", content)

        if isinstance(result, str):
            result = [result]
        result = [r.strip() for r in result if r.strip()]
        result = list(set(result))

    except Exception as e:
        print(f"Error processing description: {e}")
        result = []

    return result


# ------------------------------------------------------
# 🔁 Apply once per unique video_id
# ------------------------------------------------------

def assign_guest_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply guest extraction once per unique video_id.
    Adds a 'guest_list' column to the DataFrame.
    """
    # Create mapping: video_id → guest list
    mapping = {}
    unique_videos = df.drop_duplicates(subset="video_id")[["video_id", "video_description"]]

    for _, row in unique_videos.iterrows():
        vid = row["video_id"]
        desc = row["video_description"]
        guests = get_guest_names_openrouter(desc)
        mapping[vid] = guests

    # Map results back to main DataFrame
    df["guest_list"] = df["video_id"].map(mapping)
    return df

In [6]:
df = assign_guest_names(df)

In [7]:
df['guest_list'].value_counts()

guest_list
[Dr Benjamin Bikman]             38
[Steven, Vanessa Van Edwards]    22
[Dr Jordan Peterson]             15
[Master Shi Heng Yi]              7
[Dr Stacy Sims]                   7
[Dr Peter Attia]                  6
[Emma Grede]                      3
[Dr. Sara Szal]                   2
Name: count, dtype: int64

In [8]:
df.head(2)

,channel_name,channel_id,video_id,video_title,video_description,video_published_at,view_count,video_like_count,comment_count,comment_id,comment_text,author,author_id,comment_like_count,comment_published_at,is_pinned,is_reply,parent_comment_id,guest_list
77400,The Diary Of A CEO,UCGq-a57w-aPwyi3pW7XLiHw,NyFSkGMWP5Q,The Insulin & Glucose Doctor: This Will Strip ...,"88% of adults have metabolic disease, but what...",2025-02-06T08:00:17Z,5387358,162020,10597,UgyU9rxYG4P0NUL8WA54AaABAg.AJJ7oTBGcPJAM8MkWBCtqL,Great to hear about your turn around. Bravo!,@roflorida1,UC7q8r-7ahW4TtkoUziiQoww,0,2025-08-22T19:50:30Z,False,True,UgyU9rxYG4P0NUL8WA54AaABAg,[Dr Benjamin Bikman]
58070,The Diary Of A CEO,UCGq-a57w-aPwyi3pW7XLiHw,Hik6OY-nk4c,Jordan B Peterson: You Need To Listen To Your ...,Dr Jordan Peterson is a world-renowned former ...,2025-01-13T08:00:19Z,2046763,54184,6392,UgyXAbAEOln-oBI6w4h4AaABAg.ADEVDGzRuVDADEiCiG23KF,Not a very open minded view....,@NiniM8154,UCLyDLLBemSNUmPREQpFGr5A,1,2025-01-13T14:19:37Z,False,True,UgyXAbAEOln-oBI6w4h4AaABAg,[Dr Jordan Peterson]


In [9]:
df['video_description'].values[0]

'88% of adults have metabolic disease, but what’s really making us sick? Dr Benjamin Bikman reveals the hidden dangers of insulin and how to take control of your health  \xa0Dr Benjamin Bikman is a metabolic scientist and Professor of Cell Biology and Physiology. He is the host of ‘The Metabolic Classroom’ podcast and author of books such as, ‘Why We Get Sick: The Hidden Epidemic at the Root of Most Chronic Disease―and How to Fight It’.\xa0  00:00 Intro 02:19 My Mission to Help with Chronic Diseases 05'

## Text Cleaning

In [10]:
import re
import spacy
import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.cluster import KMeans

# ------------------------------------------------------
# 1. Text Cleaning (light only)
# ------------------------------------------------------
def clean_comment(text: str) -> str:
    """
    Light cleaning for comments:
    - Remove @mentions
    - Remove URLs
    - Remove emojis / non-ascii
    - Lowercase
    - Strip whitespace
    - Keep context words (no lemmatization, no stopword removal yet)
    """
    if not isinstance(text, str):
        return ""

    # remove mentions
    text = re.sub(r"@\w+", "", text)

    # remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)

    # remove emojis/non-ascii
    text = text.encode("ascii", "ignore").decode()

    # lowercase + strip
    text = text.lower().strip()

    return text


In [11]:
# Apply light cleaning to the comment_text column
df["cleaned_text"] = df["comment_text"].apply(clean_comment)

# Preview the results
pd.set_option('display.max_colwidth', 100)
df[["comment_text", "cleaned_text"]].head(10)

,comment_text,cleaned_text
77400,Great to hear about your turn around. Bravo!,great to hear about your turn around. bravo!
58070,Not a very open minded view....,not a very open minded view....
49748,In many countries 😢,in many countries
82498,1 hour and 43 minutes. He refferences biggest loser and he says they gain it all back. Which is ...,1 hour and 43 minutes. he refferences biggest loser and he says they gain it all back. which is ...
75284,This channel needs to catch up with the world and what matters the most!,this channel needs to catch up with the world and what matters the most!
65006,I'm happy for you \n💕❤️💕❤️💕❤️💕\n✅✅✅✅✅✅✅\n😍😍😍😍😍😍😍,i'm happy for you
47985,Peter said 15% die within a year - not 50%. Click bait?,peter said 15% die within a year - not 50%. click bait?
78134,"Is that for us to answer? If you had to ask and it's for you, I guess you know the answer 🤦🏾‍♀️","is that for us to answer? if you had to ask and it's for you, i guess you know the answer"
51625,Literally proves her point about an employee mindset....,literally proves her point about an employee mindset....
77701,the brain needs cholesterol. \ngreat guest. ty,the brain needs cholesterol. \ngreat guest. ty


## Topic category (llama) -> Once per video

In [12]:
import ollama
import json

def get_topic_category(title: str) -> str:
    """
    Use Ollama to classify the video title into a topic category.
    Example categories: health, mental health, productivity, finance, relationships, entrepreneurship, other.
    """
    prompt = f"""
    You are a helpful assistant. Categorize the following YouTube video title into ONE broad category:
    - health
    - mental health / psychology
    - productivity / personal development
    - finance
    - relationships
    - entrepreneurship / business
    - Religion / Spirituality
    - Technology
    - Education
    - Lifestyle
    - Entertainment
    - other

    Title: "{title}"

    Return only the category name, nothing else.
    """

    response = ollama.chat(
        model="llama3.2:3b",  # you can swap to another local model
        messages=[{"role": "user", "content": prompt}]
    )
    return response["message"]["content"].strip().lower()


In [13]:
# Enable tqdm for pandas
tqdm.pandas()

# Extract unique video_id/title pairs
video_meta = df[["video_id", "video_title"]].drop_duplicates()

# Apply Ollama category classification
video_meta["Topic_Category"] = video_meta["video_title"].apply(get_topic_category)

# Merge back into main dataframe
df = df.merge(video_meta[["video_id", "Topic_Category"]], on="video_id", how="left")

In [14]:
df['Topic_Category'].value_counts()

Topic_Category
health                                 53
personal development                   22
mental health / psychology             15
other                                   7
productivity / personal development     3
Name: count, dtype: int64

## Sentiment/Comment Analysis

In [15]:
# ===========================
# Fast, Scalable Comment NLP
# ===========================

# - Batched RoBERTa inference (vectorized on CPU/GPU)
# - Parallel LLM calls (Gemma) for sentiment + guest/topic when used
# - Optional SQLite caching (sentiment & meta) to skip repeat work
# - Batched spaCy with nlp.pipe for names in comments/titles
#
# - We still call the LLM for sentiment on EVERY row (no gating)
# - Guest logic: PERSON anywhere in text -> accept; else LLM only if hints; else title fallback
# - Topic logic: only if topic-hint -> LLM extraction (no proximity filters)
# - Same ensemble weights and heuristics

import os
import re
import json
import sqlite3
import hashlib
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
from tqdm import tqdm

import spacy
import ollama

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tqdm.pandas()

# -----------------------------
# 0) Helpers & resources
# -----------------------------
NEG_PHRASES = [
    r"\bnot good\b", r"\bnot great\b", r"\bnot helpful\b", r"\bdon't like\b", r"\bdont like\b",
    r"\bnot worth\b", r"\bwaste of time\b", r"\btoo long\b", r"\btoo slow\b", r"\btoo loud\b",
    r"\bmisleading\b", r"\bclickbait\b", r"\bbiased\b", r"\bconfusing\b", r"\bannoying\b",
    r"\bcringe\b", r"\bstupid\b", r"\bdumb\b", r"\bbad\b", r"\bawful\b", r"\bterrible\b",
    r"\buseless\b", r"\bpointless\b", r"\bwrong\b", r"\bpoor (audio|sound|quality)\b",
    r"\birrelevant\b", r"\boff\-topic\b", r"\bproblem\b", r"\bissue\b", r"\bdisappoint(ing|ed)\b",
    r"\brude\b", r"\boffensive\b", r"\bunfunny\b", r"\bboring\b", r"\blazy\b", r"\btoxic\b",
    r"\bhate\b", r"\bgarbage\b", r"\bignorant\b", r"\bweird\b", r"\bnegative\b", r"\bbroken\b",
    r"\bdownvote\b", r"\bterribly\b", r"\bdislike\b", r"\bpathetic\b", r"\bworse\b"
]
NEG_RE = re.compile("|".join(NEG_PHRASES), re.I)
SHORT_PRAISE_RE = re.compile(r"^(nice|cool|great|good|amazing|awesome|wow|love|thanks|perfect)[.!]?$", re.I)

# Hints for guest & topic LLM calls (same as Code 1)
GUEST_HINTS = re.compile(r"(with|feat\.|featuring|guest|bring back|have .* on|invite|episode with|talk to|feature)", re.I)
TITLE_CUE_RE = re.compile(r"feat\.|featuring|with|guest|w/", re.I)
TOPIC_HINTS = re.compile(r"(talk about|discuss|more on|more of|do one about|episode on|video on|cover|speak about|conversation on|should do|would love|should talk about|content about)", re.I)

# Light stoplist for junky "names" (same spirit as Code 1)
BANNED_GUEST_WORDS = {
    "jesus", "praise jesus", "ohhhh", "video", "motivation", "johari", "topic", "content",
    "someone", "somebody", "anyone", "everybody"
}

# spaCy NER (as in Code 1: we only need NER; parser/tagger/lemmatizer disabled)
nlp = spacy.load("en_core_web_sm", disable=["parser", "tagger", "lemmatizer"])

def clean_comment(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"http\S+|www\S+", "", text)
    text = text.encode("ascii", "ignore").decode()
    return text.strip()

def bucket_from_p(p: float) -> str:
    if p < 0.35:
        return "Negative"
    if p > 0.65:
        return "Positive"
    return "Neutral"

def like_weight(likes: float) -> float:
    if likes is None or likes <= 0:
        return 1.0
    if likes < 10:
        return 1.2
    if likes < 100:
        return 2.0
    if likes < 500:
        return 3.0
    return 4.0

def sha1(s: str) -> str:
    return hashlib.sha1(s.encode("utf-8")).hexdigest()

# -----------------------------
# 1) HF Sentiment (RoBERTa) — BATCHED for speed (same model/logic)
# -----------------------------
_HF_MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
_tok = AutoTokenizer.from_pretrained(_HF_MODEL)
_model = AutoModelForSequenceClassification.from_pretrained(_HF_MODEL)
_model.eval()

if torch.cuda.is_available():
    DEVICE = "cuda"
elif torch.backends.mps.is_available():
    DEVICE = "mps"
else:
    DEVICE = "cpu"
_model.to(DEVICE)

@torch.no_grad()
def roberta_probs_batch(texts: List[str]) -> np.ndarray:
    """
    Vectorized version of Code 1's single-text call.
    - We still trim very long strings (char-level) to ~512 to mimic Code 1's behavior.
    - We enable truncation to model_max_length for safety when token count overruns.
    Returns array shape (N, 3) = [neg, neu, pos] per row.
    """
    if not texts:
        return np.zeros((0, 3), dtype=float)
    chopped = [(t[:512] if isinstance(t, str) else "") for t in texts]
    probs_all = []
    bs = 256  # tune as needed
    for i in range(0, len(chopped), bs):
        batch = chopped[i:i + bs]
        enc = _tok(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=min(512, _tok.model_max_length),
        )
        enc = {k: v.to(DEVICE) for k, v in enc.items()}
        logits = _model(**enc).logits
        probs = F.softmax(logits, dim=1).cpu().numpy()
        probs_all.append(probs)
    return np.vstack(probs_all)

# -----------------------------
# 2) LLM Sentiment (Gemma 2B) — PARALLEL for speed (same prompt/heuristics)
# -----------------------------
def _gemma_sent_prompt(text: str) -> str:
    return f"""
You are a neutral linguistic expert analyzing sentiment.
Evaluate only the tone of the YouTube comment.
Consider sarcasm and negation carefully.
Return JSON only: {{"score": <float between -1.0 and 1.0>}}
Comment: "{text}"
""".strip()

def gemma_score_01(text: str, model_name: str = "gemma:2b") -> float:
    if not text:
        return 0.5
    try:
        resp = ollama.chat(model=model_name, messages=[{"role":"user","content":_gemma_sent_prompt(text)}])
        j = json.loads(resp["message"]["content"])
        s = float(j.get("score", 0.0))
    except Exception:
        s = 0.0

    p = (s + 1.0) / 2.0
    if NEG_RE.search(text) and p > 0.5:
        p -= 0.25
    if SHORT_PRAISE_RE.match(text) and p > 0.7:
        p = 0.6
    return float(np.clip(p, 0.0, 1.0))

def parallel_llm_scores(texts: List[str], max_workers: int = 8, model_name: str = "gemma:2b") -> List[float]:
    scores = [0.5] * len(texts)
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = {ex.submit(gemma_score_01, t, model_name): i for i, t in enumerate(texts)}
        for fut in tqdm(as_completed(futs), total=len(futs), desc="LLM sentiment (parallel)"):
            i = futs[fut]
            try:
                scores[i] = fut.result()
            except Exception:
                scores[i] = 0.5
    return scores

# -----------------------------
# 3) Ensemble Sentiment — SAME MATH as Code 1, but batched
# -----------------------------
def ensemble_sentiment_batch(texts: List[str],
                             llm_workers: int = 8,
                             llm_model: str = "gemma:2b") -> Dict[str, np.ndarray]:
    cleaned = [clean_comment((t or "").lower()) for t in texts]
    # HF (vectorized)
    hf_probs = roberta_probs_batch(cleaned)
    if hf_probs.shape[0] == 0:
        n = len(texts)
        return dict(
            p_pos=np.full(n, 0.5),
            bucket=np.array(["Neutral"] * n, dtype=object),
            p_pos_llm=np.full(n, 0.5),
            p_pos_hf=np.full(n, 1/3),
            p_neg_hf=np.full(n, 1/3),
        )
    p_neg_hf, p_neu_hf, p_pos_hf = hf_probs[:, 0], hf_probs[:, 1], hf_probs[:, 2]

    # LLM (parallel for EVERY row — just like Code 1)
    p_pos_llm = np.array(parallel_llm_scores(cleaned, max_workers=llm_workers, model_name=llm_model), dtype=float)

    # Same blend & penalty as Code 1
    w_llm, w_hf = 0.4, 0.6
    p_pos = w_llm * p_pos_llm + w_hf * p_pos_hf
    mask_penalty = (p_neg_hf - p_pos_hf) > 0.20
    p_pos[mask_penalty] = np.maximum(0.0, p_pos[mask_penalty] - 0.20)
    p_pos = np.clip(p_pos, 0.0, 1.0)

    # Buckets (vectorized)
    bucket = np.empty(len(p_pos), dtype=object)
    bucket[p_pos < 0.35] = "Negative"
    bucket[p_pos > 0.65] = "Positive"
    bucket[(p_pos >= 0.35) & (p_pos <= 0.65)] = "Neutral"

    return dict(
        p_pos=p_pos,
        bucket=bucket,
        p_pos_llm=p_pos_llm,
        p_pos_hf=p_pos_hf,
        p_neg_hf=p_neg_hf,
    )

# -----------------------------
# 4) Caching (sentiment + meta)
# -----------------------------
CACHE_PATH = os.path.abspath("./sentiment_cache.sqlite")

def _ensure_cache():
    con = sqlite3.connect(CACHE_PATH)
    cur = con.cursor()
    cur.execute("""
      CREATE TABLE IF NOT EXISTS sentiments (
        comment_id TEXT PRIMARY KEY,
        text_hash  TEXT NOT NULL,
        p_pos      REAL NOT NULL,
        bucket     TEXT NOT NULL,
        p_pos_llm  REAL NOT NULL,
        p_pos_hf   REAL NOT NULL,
        p_neg_hf   REAL NOT NULL,
        created_at TEXT NOT NULL
      );
    """)
    cur.execute("""
      CREATE TABLE IF NOT EXISTS meta_extractions (
        comment_id  TEXT PRIMARY KEY,
        text_hash   TEXT NOT NULL,
        guest_json  TEXT NOT NULL,
        topic_json  TEXT NOT NULL,
        created_at  TEXT NOT NULL
      );
    """)
    con.commit(); con.close()

def cache_get_sentiments(comment_ids, text_hashes):
    if not comment_ids: return {}
    con = sqlite3.connect(CACHE_PATH); cur = con.cursor()
    qmarks = ",".join(["?"] * len(comment_ids))
    rows = cur.execute(f"SELECT * FROM sentiments WHERE comment_id IN ({qmarks})", comment_ids).fetchall()
    con.close()
    out = {}
    for (cid, th, p_pos, bucket, p_pos_llm, p_pos_hf, p_neg_hf, created_at) in rows:
        if text_hashes.get(cid) == th:
            out[cid] = dict(p_pos=p_pos, bucket=bucket, p_pos_llm=p_pos_llm, p_pos_hf=p_pos_hf, p_neg_hf=p_neg_hf)
    return out

def cache_put_sentiments(rows):
    if not rows: return
    con = sqlite3.connect(CACHE_PATH); cur = con.cursor()
    cur.executemany("""
      INSERT INTO sentiments (comment_id, text_hash, p_pos, bucket, p_pos_llm, p_pos_hf, p_neg_hf, created_at)
      VALUES (?, ?, ?, ?, ?, ?, ?, ?)
      ON CONFLICT(comment_id) DO UPDATE SET
        text_hash=excluded.text_hash,
        p_pos=excluded.p_pos,
        bucket=excluded.bucket,
        p_pos_llm=excluded.p_pos_llm,
        p_pos_hf=excluded.p_pos_hf,
        p_neg_hf=excluded.p_neg_hf,
        created_at=excluded.created_at;
    """, [(cid, th, ppos, buck, pll, phf, nhf, datetime.utcnow().isoformat()) for (cid, th, ppos, buck, pll, phf, nhf) in rows])
    con.commit(); con.close()

def cache_get_meta(comment_ids, text_hashes):
    if not comment_ids: return {}
    con = sqlite3.connect(CACHE_PATH); cur = con.cursor()
    qmarks = ",".join(["?"] * len(comment_ids))
    rows = cur.execute(f"SELECT * FROM meta_extractions WHERE comment_id IN ({qmarks})", comment_ids).fetchall()
    con.close()
    out = {}
    for (cid, th, gj, tj, created_at) in rows:
        if text_hashes.get(cid) == th:
            out[cid] = {"guest_mentions": json.loads(gj), "topic_requests": json.loads(tj)}
    return out

def cache_put_meta(rows):
    if not rows: return
    con = sqlite3.connect(CACHE_PATH); cur = con.cursor()
    cur.executemany("""
      INSERT INTO meta_extractions (comment_id, text_hash, guest_json, topic_json, created_at)
      VALUES (?, ?, ?, ?, ?)
      ON CONFLICT(comment_id) DO UPDATE SET
        text_hash=excluded.text_hash,
        guest_json=excluded.guest_json,
        topic_json=excluded.topic_json,
        created_at=excluded.created_at;
    """, [(cid, th, json.dumps(guests), json.dumps(topics), datetime.utcnow().isoformat()) for (cid, th, guests, topics) in rows])
    con.commit(); con.close()

# -----------------------------
# 5) Guest Extraction — SAME LOGIC as Code 1, but batched
# -----------------------------
def _llm_guest_prompt(text: str) -> str:
    return f"""
You are analyzing a YouTube comment on a podcast episode.

Your goal is to extract only the names of people the viewer *explicitly or implicitly wants to appear as guests* on future episodes.

Focus on:
- Mentions framed as requests or desires (e.g. “bring X on”, “invite Y”, “you should talk to Z”, “have A back”).
- Include both full names and recognizable single names or titles (e.g. “Dr. Huberman”, “Peterson”, “Elon Musk”).
- Ignore names mentioned for other reasons (examples, comparisons, stories).
- Exclude religious figures, fictional characters, vague references, or non-human entities.

Return strictly a JSON array of names (no explanation, no duplicates, no empty entries).

Example:
["Jordan Peterson", "Dr. Rhonda Patrick", "Elon Musk"]

Comment: "{text}"
""".strip()

def _llm_extract_guests_single(text: str, model_name: str = "gemma:2b") -> List[str]:
    try:
        resp = ollama.chat(model=model_name, messages=[{"role":"user","content":_llm_guest_prompt(text)}])
        names = json.loads(resp["message"]["content"])
    except Exception:
        names = []

    if isinstance(names, list):
        names = [
            n.strip() for n in names
            if n and len(n) > 2 and n[0].isupper() and n.lower() not in BANNED_GUEST_WORDS
        ]
    else:
        names = []
    return list(dict.fromkeys(names))  # dedupe, keep order

def extract_guests_batch(texts: List[str], titles: Optional[List[str]] = None,
                         max_workers: int = 8, llm_model: str = "gemma:2b") -> List[List[str]]:
    """
    Code 1 flow, just faster:
      1) spaCy PERSON anywhere in text → accept immediately (light filter)
      2) If none and GUEST_HINTS present → parallel LLM extraction
      3) If still none and title has "feat./with/guest" → spaCy PERSON from title
    """
    titles = titles or [""] * len(texts)
    cleaned = [clean_comment(t) for t in texts]
    results = [[] for _ in cleaned]

    # 1) spaCy PERSON anywhere (batched)
    docs = list(nlp.pipe(cleaned, batch_size=256, n_process=1))
    for i, doc in enumerate(docs):
        cand = [ent.text.strip() for ent in doc.ents if ent.label_ == "PERSON"]
        if cand:
            cand = [n for n in cand if len(n) > 2 and n.lower() not in BANNED_GUEST_WORDS]
            if cand:
                results[i] = list(dict.fromkeys(cand))  # keep order

    # 2) LLM for rows with hints but still empty
    llm_idxs = [i for i, (t, r) in enumerate(zip(cleaned, results)) if not r and GUEST_HINTS.search(t or "")]
    if llm_idxs:
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futs = {ex.submit(_llm_extract_guests_single, cleaned[i], llm_model): i for i in llm_idxs}
            for fut in tqdm(as_completed(futs), total=len(futs), desc="Guests LLM (parallel)"):
                i = futs[fut]
                try:
                    results[i] = fut.result()
                except Exception:
                    results[i] = []

    # 3) Title fallback
    fallback_idxs = [i for i in range(len(cleaned)) if not results[i] and TITLE_CUE_RE.search(titles[i] or "")]
    if fallback_idxs:
        title_docs = list(nlp.pipe([titles[i] for i in fallback_idxs], batch_size=256, n_process=1))
        for i, doc in zip(fallback_idxs, title_docs):
            cand = [ent.text.strip() for ent in doc.ents if ent.label_ == "PERSON"]
            cand = [n for n in cand if len(n) > 2 and n.lower() not in BANNED_GUEST_WORDS]
            results[i] = list(dict.fromkeys(cand))

    return results

# -----------------------------
# 6) Topic Request Extraction — SAME LOGIC as Code 1, but parallel
# -----------------------------
def _llm_topic_prompt(text: str) -> str:
    return f"""
You are analyzing a YouTube comment on a podcast episode.

Your goal is to extract *specific topics or subjects the viewer explicitly wants discussed in future episodes.*

Focus on:
- Requests or suggestions like “talk about X”, “do an episode on Y”, “discuss more about Z”, “I’d love a conversation on…”.
- Extract concise, meaningful topics (1–4 words) representing what the viewer wants to hear more about.
- Ignore vague mentions, compliments, or generic categories (like “science”, “health”, “AI”) unless clearly framed as requests.
- Exclude random nouns or things already being discussed.

Return strictly a JSON list of short topic phrases (no explanation, no duplicates).

Example:
["mental health in men", "AI and creativity", "nutrition and longevity"]

Comment: "{text}"
""".strip()

def _llm_extract_topics_single(text: str, model_name: str = "gemma:2b") -> List[str]:
    try:
        resp = ollama.chat(model=model_name, messages=[{"role":"user","content":_llm_topic_prompt(text)}])
        topics = json.loads(resp["message"]["content"])
        if isinstance(topics, list):
            topics = [str(t).strip().lower() for t in topics if str(t).strip()]
            topics = list(dict.fromkeys(topics))[:5]
            return topics
    except Exception:
        pass
    return []

def extract_topics_batch(texts: List[str], max_workers: int = 8, llm_model: str = "gemma:2b") -> List[List[str]]:
    """
    Code 1 flow, compute-friendly:
      • Only rows that match TOPIC_HINTS call the LLM (as in Code 1).
      • Parallelize the LLM calls.
    """
    cleaned = [clean_comment(t) for t in texts]
    results = [[] for _ in cleaned]
    idxs = [i for i, t in enumerate(cleaned) if t and TOPIC_HINTS.search(t)]
    if not idxs:
        return results

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = {ex.submit(_llm_extract_topics_single, cleaned[i], llm_model): i for i in idxs}
        for fut in tqdm(as_completed(futs), total=len(futs), desc="Topics LLM (parallel)"):
            i = futs[fut]
            try:
                results[i] = fut.result()
            except Exception:
                results[i] = []
    return results

# -----------------------------
# 7) Apply All to DataFrame — with caching & batching
# -----------------------------
def apply_sentiment_fast(df: pd.DataFrame,
                         text_col: str = "comment_text",
                         id_col: str = "comment_id",
                         like_col: str = "comment_like_count",
                         title_col: Optional[str] = "video_title",
                         use_cache: bool = True,
                         llm_workers: int = 8,
                         llm_model: str = "gemma:2b",
                         run_guest_topic: bool = True) -> pd.DataFrame:

    assert text_col in df.columns, f"'{text_col}' not in df"
    assert id_col in df.columns, f"'{id_col}' not in df"

    _ensure_cache()
    out = df.copy()
    out[text_col] = out[text_col].fillna("").astype(str)
    out["cleaned_text"] = out[text_col].map(clean_comment)

    # sentiment cache prep
    out["__text_hash"] = out["cleaned_text"].map(sha1)
    ids = out[id_col].astype(str).tolist()
    id_to_hash = dict(zip(ids, out["__text_hash"].tolist()))
    cached_sent = cache_get_sentiments(ids, id_to_hash) if use_cache else {}
    cached_mask_sent = out[id_col].astype(str).isin(cached_sent.keys())

    # init cols
    for col in ["sentiment_p_pos", "sentiment_bucket", "p_pos_llm", "p_pos_hf", "p_neg_hf"]:
        out[col] = np.nan if col != "sentiment_bucket" else None

    # fill from cache
    if cached_sent:
        idx = out.index[cached_mask_sent]
        for i in idx:
            cid = str(out.at[i, id_col]); row = cached_sent[cid]
            out.at[i, "sentiment_p_pos"] = row["p_pos"]
            out.at[i, "sentiment_bucket"] = row["bucket"]
            out.at[i, "p_pos_llm"] = row["p_pos_llm"]
            out.at[i, "p_pos_hf"] = row["p_pos_hf"]
            out.at[i, "p_neg_hf"] = row["p_neg_hf"]

    # compute missing (batched HF + parallel LLM, SAME LOGIC)
    need = out.index[~cached_mask_sent].tolist()
    if need:
        texts = out.loc[need, "cleaned_text"].tolist()
        scores = ensemble_sentiment_batch(texts, llm_workers=llm_workers, llm_model=llm_model)
        out.loc[need, "p_pos_hf"] = scores["p_pos_hf"]
        out.loc[need, "p_neg_hf"] = scores["p_neg_hf"]
        out.loc[need, "p_pos_llm"] = scores["p_pos_llm"]
        out.loc[need, "sentiment_p_pos"] = scores["p_pos"]
        out.loc[need, "sentiment_bucket"] = scores["bucket"]

        if use_cache:
            to_cache = []
            for i in need:
                to_cache.append((
                    str(out.at[i, id_col]),
                    out.at[i, "__text_hash"],
                    float(out.at[i, "sentiment_p_pos"]),
                    str(out.at[i, "sentiment_bucket"]),
                    float(out.at[i, "p_pos_llm"]),
                    float(out.at[i, "p_pos_hf"]),
                    float(out.at[i, "p_neg_hf"]),
                ))
            cache_put_sentiments(to_cache)

    # impact weighting
    if like_col in out.columns:
        out[like_col] = out[like_col].fillna(0).astype(int)
        out["comment_weight"] = out[like_col].map(like_weight)
    else:
        out["comment_weight"] = 1.0
    out["impact_weighted_sentiment"] = out["sentiment_p_pos"].astype(float) * out["comment_weight"].astype(float)

    # guests/topics (same logic, parallelized + cached)
    if run_guest_topic:
        cached_meta = cache_get_meta(ids, id_to_hash) if use_cache else {}
        cached_mask_meta = out[id_col].astype(str).isin(cached_meta.keys())
        out["guest_mentions"] = [[] for _ in range(len(out))]
        out["topic_requests"] = [[] for _ in range(len(out))]

        # from cache
        if cached_meta:
            idx = out.index[cached_mask_meta]
            for i in idx:
                cid = str(out.at[i, id_col]); row = cached_meta[cid]
                out.at[i, "guest_mentions"] = row["guest_mentions"]
                out.at[i, "topic_requests"] = row["topic_requests"]

        # compute missing
        need_meta = out.index[~cached_mask_meta].tolist()
        if need_meta:
            texts = out.loc[need_meta, "cleaned_text"].tolist()
            titles = out[title_col].tolist() if title_col and title_col in out.columns else [""] * len(out)

            guests_lists = extract_guests_batch(texts, [titles[i] for i in need_meta],
                                                max_workers=llm_workers, llm_model=llm_model)
            topics_lists = extract_topics_batch(texts, max_workers=llm_workers, llm_model=llm_model)

            for i, glst in zip(need_meta, guests_lists):
                out.at[i, "guest_mentions"] = glst
            for i, tlst in zip(need_meta, topics_lists):
                out.at[i, "topic_requests"] = tlst

            if use_cache:
                to_meta = []
                for i in need_meta:
                    to_meta.append((
                        str(out.at[i, id_col]),
                        out.at[i, "__text_hash"],
                        out.at[i, "guest_mentions"],
                        out.at[i, "topic_requests"],
                    ))
                cache_put_meta(to_meta)

    out.drop(columns=["__text_hash"], inplace=True, errors="ignore")
    return out

# ===========================
# Example
# ===========================
# out = apply_sentiment_fast(
#     df,
#     text_col="comment_text",
#     id_col="comment_id",
#     like_col="comment_like_count",
#     title_col="video_title",
#     use_cache=True,          # set False if you want fresh runs every time
#     llm_workers=8,           # increase if you have more CPU cores / IO
#     llm_model="gemma:2b",
#     run_guest_topic=True
# )


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
df = apply_sentiment_fast(
    df,
    text_col="comment_text",
    id_col="comment_id",
    like_col="comment_like_count",
    title_col="video_title",
    use_cache=True,          # set False if you want fresh runs every time
    llm_workers=8,           # increase if you have more CPU cores / IO
    llm_model="gemma:2b",
    run_guest_topic=True
)

LLM sentiment (parallel):   0%|          | 0/96 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Topics LLM (parallel): 100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


In [17]:
df[['video_title', 'guest_list','Topic_Category','cleaned_text', 'sentiment_p_pos', 'sentiment_bucket', 'p_pos_llm', 'p_pos_hf', 'p_neg_hf', 'comment_like_count', 'comment_weight', 'impact_weighted_sentiment']].head(10)

,video_title,guest_list,Topic_Category,cleaned_text,sentiment_p_pos,sentiment_bucket,p_pos_llm,p_pos_hf,p_neg_hf,comment_like_count,comment_weight,impact_weighted_sentiment
0,The Insulin & Glucose Doctor: This Will Strip Your Fat Faster Than Anything!,[Dr Benjamin Bikman],health,Great to hear about your turn around. Bravo!,0.926439,Positive,0.835,0.987398,0.003098,0,1.0,0.926439
1,Jordan B Peterson: You Need To Listen To Your Wife! We've Built A Lonely & Sexless Society!,[Dr Jordan Peterson],mental health / psychology,Not a very open minded view....,0.110327,Negative,0.750,0.017211,0.668515,1,1.2,0.132392
2,"Hormone Expert: Control Your Hormones Control Your Belly Fat! Cortisol, oestrogen, testosterone.",[Dr. Sara Szal],health,In many countries,0.336931,Negative,0.750,0.061551,0.082644,0,1.0,0.336931
3,The Insulin & Glucose Doctor: This Will Strip Your Fat Faster Than Anything!,[Dr Benjamin Bikman],health,1 hour and 43 minutes. He refferences biggest loser and he says they gain it all back. Which is ...,0.145619,Negative,0.850,0.009365,0.802823,0,1.0,0.145619
4,The Insulin & Glucose Doctor: This Will Strip Your Fat Faster Than Anything!,[Dr Benjamin Bikman],health,This channel needs to catch up with the world and what matters the most!,0.308528,Negative,0.500,0.180879,0.126774,1,1.2,0.370233
5,Shaolin Warrior Master: Hidden Epidemic Nobody Talks About! This Modern Habit Is Killing Millions!,[Master Shi Heng Yi],other,I'm happy for you,0.920455,Positive,0.830,0.980759,0.003789,0,1.0,0.920455
6,Peter Attia: Anti-aging Cure No One Talks About! 50% Chance You’ll Die In A Year If This Happens!,[Dr Peter Attia],health,Peter said 15% die within a year - not 50%. Click bait?,0.105774,Negative,0.750,0.009624,0.682345,0,1.0,0.105774
7,The Insulin & Glucose Doctor: This Will Strip Your Fat Faster Than Anything!,[Dr Benjamin Bikman],health,"Is that for us to answer? If you had to ask and it's for you, I guess you know the answer",0.346968,Negative,0.750,0.078279,0.097291,0,1.0,0.346968
8,The Business Expert: How To Build A Brand In 2025! They're Lying To You About Work-Life Balance!,[Emma Grede],productivity / personal development,Literally proves her point about an employee mindset....,0.421810,Neutral,0.835,0.146351,0.099479,1,1.2,0.506172
9,The Insulin & Glucose Doctor: This Will Strip Your Fat Faster Than Anything!,[Dr Benjamin Bikman],health,the brain needs cholesterol. \ngreat guest. ty,0.674951,Positive,0.675,0.674918,0.050141,0,1.0,0.674951


In [18]:
print(df['sentiment_bucket'].value_counts())
print("\n")
print(df['comment_weight'].value_counts())
print("\n")
print(df['guest_mentions'].value_counts())
print("\n")
print(df['topic_requests'].value_counts())

sentiment_bucket
Negative    56
Positive    29
Neutral     15
Name: count, dtype: int64


comment_weight
1.0    69
1.2    28
2.0     3
Name: count, dtype: int64


guest_mentions
[]                                                  82
[Ben Bikman]                                         2
[Doc]                                                1
[Steven]                                             1
[Keto]                                               1
[Steven, Tim Spector - a]                            1
[Jordan Peterson, Dr. Rhonda Patrick, Elon Musk]     1
[Dr. Rhonda Patrick, Elon Musk]                      1
[Jordan Peterson]                                    1
[Benjamin Bikman]                                    1
[Jillian michaels]                                   1
[Bernstein]                                          1
[Joe Rogan]                                          1
[Bendiciones]                                        1
[Jordan B Peterson, Steven]                         

In [24]:
df.groupby('video_title')[['sentiment_bucket']].value_counts()

video_title                                                                                         sentiment_bucket
Body Language Expert Explains Why People Dislike You                                                Negative            16
                                                                                                    Positive             5
                                                                                                    Neutral              1
Exercise & Nutrition Scientist: The Truth About Exercise On Your Period! Take These 4 Supplements!  Negative             3
                                                                                                    Neutral              3
                                                                                                    Positive             1
Hormone Expert: Control Your Hormones Control Your Belly Fat! Cortisol, oestrogen, testosterone.    Negative             2
Jordan B Peterson: You

# What's missing or could be improved?

- How to best score sentiment against the topic or guests or both and give them weights that make sense
- an LLM that goes over the guest_mentions and topic_requests to clean and normalize them (GPT4 should do)
- Visualise results in a way that is interesting and helps make informed decisions about future guests/topics